In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [22]:
# 장비 할당하기
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'
BATCH_SIZE = 64
Epochs = 20

print("torch version : {}".format(torch.__version__))
print("torch Device available : {}".format(DEVICE))
print("train Batch_size : {}\ntrani Epochs : {}".format(BATCH_SIZE,Epochs))

torch version : 1.7.1+cu110
torch Device available : cuda
train Batch_size : 64
trani Epochs : 20


In [23]:
# CIFAR100 데이터 불러오기
train_datasets = datasets.CIFAR100(root = './Data/',download = True, train = True, transform = transforms.ToTensor())
test_datasets =  datasets.CIFAR100(root = './Data/',download = True, train = False,transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_datasets, batch_size = BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size = BATCH_SIZE, shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


In [24]:
# 지정한 배치사이즈로 내가 원하는 데이터가 불러와지는지 확인
# 입력 차원 확인하기
for (image, label) in train_loader:
    print("image size : {}".format(image.size()))
    print("label size : {}".format(label.size()))
    break

image size : torch.Size([64, 3, 32, 32])
label size : torch.Size([64])


<img src = './images/resnet.png' width = 1000px>

In [75]:
class BasicBlock(nn.Module):
    def __init__(self,in_planes,planes,stride = 1):
        super(BasicBlock,self).__init__()

        self.conv1 = nn.Conv2d(in_planes,planes,kernel_size=3,stride=stride,padding=1,bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes,planes,kernel_size=3,stride=1,padding=1,bias = False)
        self.bn2 = nn.BatchNorm2d(planes)

        # short_cut 의 역할은 stride가 달라 차원이 달라진 경우, kernel_size는 1로 고정하고 stride값을 조정해 차원을 맞춰주는 역할이다.
        self.short_cut = nn.Sequential()
        if stride != 1:
            self.short_cut = nn.Sequential(
                nn.Conv2d(in_planes,planes,kernel_size=1,stride=stride, bias = False),
                nn.BatchNorm2d(planes)
            )

    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.short_cut(x)
        out = F.relu(out)
        return out

In [86]:
class ResNet18(nn.Module):
    def __init__(self, num_classes = 100):
        super(ResNet18,self).__init__()
        self.blocks = [2,2,2,2]
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3,64,kernel_size=3,stride=2,bias = False)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)

        # layer0 의 stride 값이 1인 이유는 바로 위, MaxPool2d를 통해 stride값을 주며 이미 출력이미지에 맞춰서 넘어왔기 때문이다.
        # 나머지 layer sequence 들은 다음 block으로 넘어갈 때 출력이미지가 절반으로 줄어들기 때문에 stride = 2 값을 준다.
        self.layer1 = self.make_layer(64,self.blocks[0],stride=1)
        self.layer2 = self.make_layer(128,self.blocks[1],stride=2)
        self.layer3 = self.make_layer(256,self.blocks[2],stride=2)
        self.layer4 = self.make_layer(512,self.blocks[3],stride=2)

        self.fc_layer = nn.Linear(512,100)
    
    def make_layer(self,planes,block,stride):
        strides = [stride] + block * [1]

        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes,planes,stride))
            self.in_planes = planes
        
        return nn.Sequential(* layers)
    
    def forward(self,x):
        out = F.relu(self.pool1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        # Feature Map에 4*4 filter가 움직이면서 16개의 Feature Map 값의 평균을 계산해 1개의 Feature Map으로 다운샘플링
        # out = F.avg_pool2d(out,8)
        out = out.view(out.size(0),-1)
        out = self.fc_layer(out)

        return out

In [87]:
model = ResNet18().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr =0.01)
criterion = nn.CrossEntropyLoss()

print(model)

ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (short_cut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [88]:
def train(model, train_loader, optimizer, log_interval):
    model.train()

    for batch_idx,(image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch : {} [ {} / {} ( {:.0f}% ) ]'.format(Epoch, batch_idx * len(image), len(train_loader.dataset), 100 * batch_idx / len(train_loader)), end = ',  ')
            print('Train Loss : {:.6f}'.format(loss.item()))

In [89]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)

            output = model(image)
            test_loss += criterion(output,label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
        
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100 * correct / len(test_loader.dataset)

        return test_loss, test_accuracy

In [90]:
for Epoch in range(1, Epochs+1):
    train(model, train_loader, optimizer, 200)
    test_loss, test_accracy = evaluate(model, test_loader)
    print("\n[Epoch : {}], \t Test Loss : {:.4f}, \t Test Accuracy : {:.2f} %\n".format(Epoch, test_loss, test_accracy))

Train Epoch : 1 [ 0 / 50000 ( 0% ) ],  Train Loss : 4.993938
Train Epoch : 1 [ 12800 / 50000 ( 26% ) ],  Train Loss : 4.306392
Train Epoch : 1 [ 25600 / 50000 ( 51% ) ],  Train Loss : 4.106871
Train Epoch : 1 [ 38400 / 50000 ( 77% ) ],  Train Loss : 3.732512

[Epoch : 1], 	 Test Loss : 0.0620, 	 Test Accuracy : 7.84 %

Train Epoch : 2 [ 0 / 50000 ( 0% ) ],  Train Loss : 3.793797
Train Epoch : 2 [ 12800 / 50000 ( 26% ) ],  Train Loss : 3.776159
Train Epoch : 2 [ 25600 / 50000 ( 51% ) ],  Train Loss : 3.520288
Train Epoch : 2 [ 38400 / 50000 ( 77% ) ],  Train Loss : 3.731096

[Epoch : 2], 	 Test Loss : 0.0592, 	 Test Accuracy : 11.76 %

Train Epoch : 3 [ 0 / 50000 ( 0% ) ],  Train Loss : 3.532098
Train Epoch : 3 [ 12800 / 50000 ( 26% ) ],  Train Loss : 3.674908
Train Epoch : 3 [ 25600 / 50000 ( 51% ) ],  Train Loss : 3.512073
Train Epoch : 3 [ 38400 / 50000 ( 77% ) ],  Train Loss : 3.464995

[Epoch : 3], 	 Test Loss : 0.0548, 	 Test Accuracy : 15.44 %

Train Epoch : 4 [ 0 / 50000 ( 0% ) 